In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
from glob import glob

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
import tensorflow as tf
from keras.models import Sequential
from keras.layers.pooling import GlobalAveragePooling2D
from tensorflow.keras.layers import Dense, Dropout
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping
import cv2 as cv
import numpy as np
import seaborn as sns
from sklearn.metrics import accuracy_score, f1_score

from keras.applications.inception_v3 import InceptionV3, preprocess_input

In [ ]:

data_list  = []


for dirname, _, filenames in os.walk(r'custom path'):# add the desire input path
    for filename in filenames:
        data_list.append (os.path.join(dirname, filename))

#using the data file we have set the path


In [ ]:
len(data_list)

In [ ]:
labels = pd.read_csv(r'custom path')
labels.head()
#csv file containing basic infos

# PLOTTING

In [ ]:

def barw(ax): 
    
    for p in ax.patches:
        val = p.get_width()
        x = p.get_x()+ p.get_width()
        y = p.get_y() + p.get_height()
        ax.annotate(round(val,2),(x,y))

plt.figure(figsize = (15,30))
ax0 =sns.countplot(y=labels['breed'],order=labels['breed'].value_counts().index)
barw(ax0)
plt.show()

In [ ]:
test_labels=['Bombay', 'Calico', 'Burmese', 'Himalayan',
'Munchkin', 'Ragdoll', 'Siberian', 'British Shorthair', 'Russian Blue', 
                 'Dilute Calico']

In [ ]:
filename= []
final_labels = []

for path in data_list:
    label = path.split(os.path.sep)[-2]
    if label in test_labels:

        filename.append(path)
        final_labels.append(label)

In [ ]:
dict.fromkeys(final_labels)

In [ ]:
len(final_labels),len(filename)#labels are now set

In [ ]:
grid=pd.DataFrame(list(zip(filename, final_labels) ), columns = ['path', 'Labels'] )


In [ ]:
grid

In [ ]:
f,a = plt.subplots(nrows=2, ncols=2,figsize=(10,20),
                        subplot_kw={'xticks': [], 'yticks': []})

for i, ax in enumerate(a.flat):
    ax.imshow(plt.imread(grid.path[i]))
    ax.set_title(grid.Labels[i])
    
plt.tight_layout()
plt.show()

# SPLITING THE DATA WITH 80/20 ratio

In [ ]:
training = .80
valid_ratio = 0.10     
testing = 0.20

train, test = train_test_split(grid, test_size = testing )
val, test = train_test_split(test, test_size=testing/(testing + valid_ratio))

In [ ]:
data_generate = ImageDataGenerator(rescale=1./255, rotation_range=30, width_shift_range=0.2,height_shift_range=0.2,horizontal_flip = 'true')
   

x_train =  data_generate.flow_from_dataframe(dataframe = train,  x_col='path', y_col='Labels',  target_size=(299, 299), shuffle=False, batch_size=10, seed=10)
x_val = data_generate.flow_from_dataframe(dataframe = val,  x_col='path', y_col='Labels',  target_size=(299, 299), shuffle=False, batch_size=10, seed=10)
x_test = data_generate.flow_from_dataframe(dataframe = test,  x_col='path', y_col='Labels',  target_size=(299, 299), shuffle=False, batch_size=10, seed=10)

In [ ]:
x_train

# USING INCEPTION MODEL

In [ ]:
finalmodel= InceptionV3(weights= 'imagenet', include_top=False, input_shape=(299, 299, 3))

In [ ]:

finalmodel.summary()

In [ ]:
cnn = Sequential()
cnn.add(finalmodel)
cnn.add(GlobalAveragePooling2D())
cnn.add(Dense(128))
cnn.add(Dropout(0.2))
cnn.add(Dense(10, activation = 'relu'))
cnn.summary()

In [ ]:


cnn.compile(optimizer= "adam", loss='mse', metrics=["accuracy"])

# TRAINING

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)
history = cnn.fit(x_train, validation_data = x_val,steps_per_epoch = 175,validation_steps = 44,
                epochs = 5, verbose = 2)

# PREDICTION

In [ ]:
predic = cnn.predict(x_test)
predic = np.argmax(predictions, axis=1)
predic

In [ ]:
final_labels = x_train.class_indices
final_labels

In [ ]:
test["Labels"].replace({"Bombay": 0,'British Shorthair': 1,
 'Burmese': 2,
 'Calico': 3,
 'Dilute Calico': 4,
 'Himalayan': 5,
 'Munchkin': 6,
 'Ragdoll': 7,
 'Russian Blue': 8,
 'Siberian': 9}, inplace = True)

In [ ]:
test_accuracy = cnn.evaluate(x_test)[1] * 100
print('accuracy: ',test_accuracy)

# CONFUSION MATRIX

In [ ]:
confusion_matrix(test.Labels , predic)

# ROC SCORE

In [ ]:
predic_percent =cnn.predict_proba(x_test)
predic_percent

# F1 SCORE

In [ ]:

print('score',f1_score(test.Labels, predic, average = 'weighted') *100, "%")